In [ ]:
import re  # Import the re module
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException,StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# List to store scraped data
list_doc = []

# Initialize the WebDriver (update to Edge if needed)
driver = webdriver.Chrome()
driver.get("https://avito.ma")
time.sleep(3)

# Function to scrape information
def scrape_information(xpath):
    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        return element.text
    except (NoSuchElementException, TimeoutException,StaleElementReferenceException):
        return "NON SPÉCIFIÉ"
        
def click_on_elements(path):
    try:
        element = WebDriverWait(driver, 5).until(  # Increased wait time for stability
            EC.element_to_be_clickable((By.XPATH, path))
        )
        element.click()
    except (NoSuchElementException, TimeoutException, ElementClickInterceptedException,StaleElementReferenceException):
        print(f"Could not click on element with path: {path}")


# function pour converter prix format
def convert_price_to_mad(price_str, exchange_rate=1):
    cleaned_price = re.sub(r'[^\d]', '', price_str)
    price_mad = int(cleaned_price)
    converted_price = price_mad * exchange_rate
    return converted_price
# Select category and confirm
click_on_elements("//*[@id='__next']/div/div[5]/div[1]/div/button[1]")
click_on_elements("//*[@id='__next']/div[2]/div[5]/div[3]/div/div[5]/div/div[4]/label")
click_on_elements("/html/body/div[1]/div[2]/div[5]/div[3]/div/div[5]/div/div[4]/div/button[1]")
click_on_elements("//*[@id='__next']/div[2]/div[5]/div[3]/div/div[5]/div/div[1]/div/button[1]")
click_on_elements("/html/body/div[1]/div[2]/div[5]/div[3]/div/div[7]/button")


# Pagination loop
page = 1
while True:
    print(f"Scraping page {page}")

    # Loop to scrape multiple listings on each page
    for i in range(1, 44):
        try:
            click_on_elements(f"//*[@id='__next']/div/main/div/div[5]/div[1]/div/div[1]/a[{i}]")
            time.sleep(3)
            current_url = driver.current_url
            # Scrape ad details
            list_doc.append({
                'titre': scrape_information("//*[@id='__next']/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/h1"),
                'prix': convert_price_to_mad(scrape_information("//*[@id='__next']/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]/p")),
                'location': scrape_information("//*[@id='__next']/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[2]/span[1]"),
                'surface': scrape_information("/html/body/div[1]/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/div/span"),
                'nombre de chambres': scrape_information("//*[@id='__next']/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[1]/div/span"),
                'étage': scrape_information("/html/body/div[1]/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[4]/div[2]/ol/li[6]/span[2]"),
                'zonage': scrape_information("//*[@id='__next']/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[4]/div/ol/li[4]/span[2]"),
                'Secteur':scrape_information("/html/body/div[1]/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[2]/div[4]/div[2]/ol/li[2]/span[2]"),
                'url': current_url
            })
            print(list_doc[-1])
            # # Go back to the previous page
            # btn_back_1("/html/body/div[1]/div/main/div/div[3]/div[1]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/button",
            #            "//*[@id='__next']/main/div[3]/div/div[1]/a/button")
            driver.back()
        except Exception as e:
            print(f"Error with listing {i} on page {page}: {e}")
            continue

    # Move to the next page
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div/main/div/div[5]/div[1]/div/div[2]/div/a[9]"))
        )
        next_button.click()
        page += 1
        time.sleep(3)  # Wait for the new page to load
    except (NoSuchElementException, TimeoutException):
        print("No more pages to scrape.")
        break
 # Sauvegarder les données dans un fichier CSV
with open('data_scraped.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = list_doc[0].keys()  # Récupérer les clés du premier élément pour les en-têtes
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(list_doc)
# Close the driver
driver.quit()
